In [9]:
import os
import random
import json
import numpy as np
import cv2
from PIL import Image, ImageDraw
from pycocotools import mask
from skimage import measure
import json

In [10]:
# Define the paths to the three folders containing the images
fg_folders = [
    ("/home/aldo/Documents/Test_Object_Detection/Starbucks_fotos_resized/","coffee" ),
    ("/home/aldo/Documents/Test_Object_Detection/Ciel_fotos_resized/", "water"),
    ("/home/aldo/Documents/Test_Object_Detection/Cel_fotos_resized/", "cellphone"),
]
bg_folder = "/home/aldo/Documents/Test_Object_Detection/backgrounds/"

In [11]:
#put manually all the categories
annotations_dict = {
    "coffee": 0,
    "water": 1,
    "cellphone": 2
}
categories=[]

for categor in annotations_dict:
    categories.append({"id": annotations_dict[categor],"name": categor,})
print(categories)

[{'id': 0, 'name': 'coffee'}, {'id': 1, 'name': 'water'}, {'id': 2, 'name': 'cellphone'}]


In [12]:
# Load the list of files in each of the three folders
fg_files = {}
for folder, category in fg_folders:
    fg_files[category] = os.listdir(folder)

In [13]:
images=[]
annotations=[]

img_id=int(0)
anno_id=int(0)
for j in range(100):
    num_objects = random.randint(0, 3)
    print("number of objects",num_objects)
    # Select random foreground images from the three folders, with replacement
    fg_categories = random.choices(["coffee", "water", "cellphone"], k=num_objects)
    fg_files_selected = []
    for category in fg_categories:
        fg_files_selected.append([category,random.choice(fg_files[category])])
    print("seleccion",fg_files_selected)
    # Load the selected foreground images using Pillow
    fg_imgs = []
    for img in fg_files_selected:
        folder = [f[0] for f in fg_folders if f[1] == img[0]][0]
        fg_imgs.append([img[0],Image.open(folder + img[1]),folder+img[1]])

    # Randomly resize, rotate, and flip the foreground images using Pillow's transform module
    for img in fg_imgs:
        fg_img=img[1]
        angle = random.randint(-45, 45)
        scale = random.uniform(0.3, 1.0)

        fg_img = fg_img.rotate(angle, resample=Image.BICUBIC, expand=True)
        fg_img = fg_img.resize((int(fg_img.width * scale), int(fg_img.height * scale)))

        img[1] = fg_img

    #change bluur randomly
    for img in fg_imgs:
        fg_img=img[1]
        blur_intensity = random.uniform(0, 1.0)*0.01
        kernel_size = (2 * round(blur_intensity * 20) + 1, 2 * round(blur_intensity * 20) + 1)
        fg_arr = np.array(fg_img)
        fg_arr = cv2.GaussianBlur(fg_arr, kernel_size, 0)
        img[1] = Image.fromarray(fg_arr)

    # Load the background image using Pillow
    bg_files = os.listdir(bg_folder)
    bg_file = random.choice(bg_files)
    bg_img = Image.open(bg_folder + bg_file)


    # Define the maximum overlap as a percentage
    max_overlap_pct = 10

    # Calculate the maximum overlap area
    max_overlap_area = (bg_img.width * bg_img.height) * (max_overlap_pct / 100)

    # Define an array to keep track of occupied areas
    occupied = np.zeros((bg_img.height, bg_img.width))

    for img in fg_imgs:
        fg_img=img[1]

        seg_img = img[1]
        # Convert the image to a NumPy array
        img_arr = np.array(seg_img)
        # Create a binary mask of the non-transparent pixels
        mask = img_arr[:, :, 3] != 0

        # Convert the mask to a COCO format segmentation
        segmentation = []
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i, j]:
                    segmentation.append(j)
                    segmentation.append(i)
        segmentation = [segmentation]

        # Calculate the area of the segmentation
        area = 0
        for i in range(len(segmentation[0]) // 2):
            x1 = segmentation[0][2 * i]
            y1 = segmentation[0][2 * i + 1]
            x2 = segmentation[0][(2 * i + 2) % len(segmentation[0])]
            y2 = segmentation[0][(2 * i + 3) % len(segmentation[0])]
            area += x1 * y2 - x2 * y1
        area = abs(area) / 2
        
        # Draw the segmentation onto a copy of the original image
        #image_copy = image.copy()
        #cv2.fillPoly(image_copy, aux_segmentation, color=(0, 255, 0))

        # Display the image with segmentation overlay
        #cv2.imshow('Image with Segmentation', image_copy)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        # Calculate the maximum allowed position for the top-left corner
        max_x = bg_img.width - fg_img.width
        max_y = bg_img.height - fg_img.height
        max_area = fg_img.width * fg_img.height

        # Generate a random location until an unoccupied area is found that meets the overlap limit

        for i in range (10):
            x = random.randint(0, max_x)
            y = random.randint(0, max_y)

            # Calculate the overlap area
            overlap_area = np.sum(occupied[y:y+fg_img.height, x:x+fg_img.width])

            # Check if the area is unoccupied and the overlap limit is not exceeded
            if (max_overlap_area - max_area) >= np.sum(occupied):
                break
            if i==10:
                continue
        for i in range(0, len(segmentation[0])):
            if i % 2:
                segmentation[0][i]=int(segmentation[0][i]+y)
            else :
                segmentation[0][i]=int(segmentation[0][i]+x)
        # Update the occupied array
        occupied[y:y+fg_img.height, x:x+fg_img.width] = 1

        bg_img.paste(fg_img, (x, y), fg_img)
        annotations.append({"id": anno_id,"image_id": img_id,"category_id": annotations_dict[img[0]],"bbox": [x, y, fg_img.width, fg_img.height],"segmentation": segmentation,"area": area,"iscrowd": 0})
        anno_id=anno_id+1
        #draw = ImageDraw.Draw(bg_img)
        #fdraw.rectangle((x, y, x+fg_img.width, y+fg_img.height), outline='red', width=3)
    bg_img.save("/home/aldo/Documents/Test_Object_Detection/output/images/"+str(img_id)+".jpg", quality=100)
    images.append({"id": img_id, "file_name": str(img_id)+".jpg","height": bg_img.height,"width": bg_img.width})
    img_id=img_id+1



number of objects 3
seleccion [['cellphone', '20230221_123131-transformed.png'], ['coffee', '20230221_123059-transformed.png'], ['water', '20230221_123149-transformed.png']]
number of objects 1
seleccion [['coffee', '20230221_123045-transformed.png']]
number of objects 0
seleccion []
number of objects 0
seleccion []
number of objects 0
seleccion []
number of objects 1
seleccion [['cellphone', '20230221_123131-transformed.png']]
number of objects 3
seleccion [['coffee', '20230221_123045-transformed.png'], ['coffee', '20230221_123054-transformed.png'], ['coffee', '20230221_123215-transformed.png']]
number of objects 2
seleccion [['cellphone', '20230221_123135-transformed.png'], ['coffee', '20230221_123213-transformed.png']]
number of objects 1
seleccion [['water', '20230221_123201-transformed.png']]
number of objects 3
seleccion [['water', '20230221_123207-transformed.png'], ['water', '20230221_123157-transformed.png'], ['cellphone', '20230221_123131-transformed.png']]
number of objects 

In [14]:
# Define the COCO dictionary
coco_dict = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

In [15]:
coco_dict

{'images': [{'id': 0, 'file_name': '0.jpg', 'height': 480, 'width': 740},
  {'id': 1, 'file_name': '1.jpg', 'height': 480, 'width': 740},
  {'id': 2, 'file_name': '2.jpg', 'height': 480, 'width': 740},
  {'id': 3, 'file_name': '3.jpg', 'height': 480, 'width': 740},
  {'id': 4, 'file_name': '4.jpg', 'height': 480, 'width': 740},
  {'id': 5, 'file_name': '5.jpg', 'height': 480, 'width': 740},
  {'id': 6, 'file_name': '6.jpg', 'height': 480, 'width': 740},
  {'id': 7, 'file_name': '7.jpg', 'height': 480, 'width': 740},
  {'id': 8, 'file_name': '8.jpg', 'height': 480, 'width': 740},
  {'id': 9, 'file_name': '9.jpg', 'height': 480, 'width': 740},
  {'id': 10, 'file_name': '10.jpg', 'height': 480, 'width': 740},
  {'id': 11, 'file_name': '11.jpg', 'height': 480, 'width': 740},
  {'id': 12, 'file_name': '12.jpg', 'height': 480, 'width': 740},
  {'id': 13, 'file_name': '13.jpg', 'height': 480, 'width': 740},
  {'id': 14, 'file_name': '14.jpg', 'height': 480, 'width': 740},
  {'id': 15, 'file_n

In [16]:
coco_dict
# Convert the dictionary to a JSON string
json_string = json.dumps(coco_dict)

# Write the JSON string to a file
with open("/home/aldo/Documents/Test_Object_Detection/output/annotations.json", 'w') as f:
    f.write(json_string)